In [1]:
import pandas as pd
data =  pd.read_csv("amazon_reviews_2M.csv",header=None)
data.head()

,0,1
0,Size Not true to size,1
1,Delicious I purchased an 8 pack of the smoked ...,20
2,They got it right! A new mystery every episode...,13
3,Wow Simply Wow!! This book was not only full o...,22
4,Love this cat I love Quincy. Don't know if the...,22


In [2]:
data.columns = ["review","rating"]


In [3]:
data.replace(to_replace=r'[^a-zA-Z\s]', value='', regex=True, inplace=True)
data.head()

,review,rating
0,Size Not true to size,1
1,Delicious I purchased an pack of the smoked p...,20
2,They got it right A new mystery every episode ...,13
3,Wow Simply Wow This book was not only full of ...,22
4,Love this cat I love Quincy Dont know if the a...,22


In [4]:
data['review'] = data['review'].str.lower().str.split()
data.head()

,review,rating
0,"[size, not, true, to, size]",1
1,"[delicious, i, purchased, an, pack, of, the, s...",20
2,"[they, got, it, right, a, new, mystery, every,...",13
3,"[wow, simply, wow, this, book, was, not, only,...",22
4,"[love, this, cat, i, love, quincy, dont, know,...",22


In [ ]:
from collections import Counter
vocab = Counter()
data['review'].apply(vocab.update)

list_of_words=[]
for word in vocab.most_common():
    if word[1]>1:
        list_of_words.append(word[0])
print(len(list_of_words))
list_of_words 

197972


['the',
 'and',
 'i',
 'a',
 'to',
 'it',
 'is',
 'of',
 'this',
 'for',
 'in',
 'my',
 'but',
 'that',
 'with',
 'great',
 'not',
 'on',
 'was',
 'have',
 'you',
 'as',
 'so',
 'are',
 'very',
 'good',
 'they',
 'its',
 'one',
 'these',
 'like',
 'be',
 'love',
 'br',
 'just',
 'them',
 'all',
 'well',
 'at',
 'if',
 'or',
 'up',
 'use',
 'will',
 'would',
 'out',
 'stars',
 'had',
 'from',
 'more',
 'when',
 'nice',
 'product',
 'me',
 'can',
 'book',
 'really',
 'has',
 'an',
 'no',
 'get',
 'what',
 'easy',
 'works',
 'than',
 'quality',
 'about',
 'time',
 'too',
 'little',
 'your',
 'perfect',
 'dont',
 'we',
 'five',
 'only',
 'do',
 'much',
 'work',
 'price',
 'there',
 'some',
 'im',
 'fit',
 'other',
 'because',
 'were',
 'her',
 'also',
 'he',
 'after',
 'bought',
 'read',
 'by',
 'even',
 'she',
 'buy',
 'which',
 'made',
 'size',
 'does',
 'used',
 'off',
 'am',
 'how',
 'small',
 'better',
 'first',
 'did',
 'been',
 'got',
 'his',
 'back',
 'way',
 'now',
 'best',
 'stil

In [ ]:
def gini_index(subset):
    class_counts = subset.value_counts()
    total = len(subset)
    gini = 1.0
    for count in class_counts:
        prob = count / total
        gini -= prob ** 2
    return gini


class TreeNode:
    def __init__(self,depth,vocab,is_root,value=None,left=None,right=None):
        self.depth = depth
        self.left = left
        self.right = right
        self.value = value
        self.vocab = vocab
        self.is_root = is_root
        self.eps = 1e-9
        
    def classify(self,features): # to reconstruct
        if self.value in features:
            return self.left.classify(features)
        else:   
            return self.right.classify(features)
        

    def split(self,dataset=None,word=None):
        left_subset = dataset[dataset['review'].apply(lambda x: word in x)]
        right_subset = dataset[dataset['review'].apply(lambda x: word not in x)]
        return left_subset,right_subset
    

    def annotate(self,dataset=None):
        # TODO Its possible to not iterate twice over gini calculation in left and right subsets 
        if isinstance(self.value,int) or isinstance(self.value,float):
            Exception("Leaf node was aleady assigned a value. Do you really wanted to annotare it again?")
        if self.depth ==10:
            self.value = dataset['rating'].mode()[0]
            self.is_root = True
            self.word = None
            
            
        Gini = gini_index(dataset['rating'])
        max_gain = -1
        for word in self.vocab:
            left_subset,right_subset = self.split(self,dataset,word)
            
            gini_left = gini_index(left_subset['rating'])
            gini_left*= (len(left_subset)/len(dataset))
            
            gini_right = gini_index(right_subset['rating'])
            gini_right*= (len(right_subset)/len(dataset))
            
            gain = Gini - (gini_left + gini_right)


            if gain>max_gain:
                max_gain = gain
                self.word = word
        if max_gain < self.eps:
            self.value = dataset['rating'].mode()[0]
            self.word = None
            self.is_root = True
            
        left_subset,right_subset = self.split(dataset,self.word)
        self.left = TreeNode(self.depth+1,self.vocab,False)
        self.right = TreeNode(self.depth+1,self.vocab,False)
        self.left.annotate(left_subset)
        self.right.annotate(right_subset)
        
class TREE_CLASSIFIER:
    def __init__(self,list_of_words,max_depth=10):
        self.V = list_of_words
        self.vocab_size = len(list_of_words)
        self.root = TreeNode(0,self.V,False)
        self.max_depth = max_depth
    def classify(self,features):
        return self.root.classify(features)

In [16]:
subset = data['rating'][~data['review'].apply(lambda x: "the" in x)]

In [19]:
subset.mode()[0]

np.int64(17)